# Cryptology Project: Classical Cipher Suite + Baudot XOR Cryptanalysis

In [1]:
import string
import math

ALPHABET = string.ascii_uppercase


In [2]:
def clean_text(text):
    """
    Keep only A–Z, convert to uppercase.
    """
    return ''.join(ch for ch in text.upper() if ch in ALPHABET)

def mod_inverse(a, m=26):
    """
    Modular inverse of a modulo m, if it exists.
    """
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 0:
            # ignore trivial 0
            continue
        if (a * x) % m == 1:
            return x
    return None


In [3]:
def caesar_encrypt(plaintext, key):
    pt = clean_text(plaintext)
    res = []
    for ch in pt:
        idx = (ALPHABET.index(ch) + key) % 26
        res.append(ALPHABET[idx])
    return ''.join(res)

def caesar_decrypt(ciphertext, key):
    ct = clean_text(ciphertext)
    res = []
    for ch in ct:
        idx = (ALPHABET.index(ch) - key) % 26
        res.append(ALPHABET[idx])
    return ''.join(res)

def caesar_bruteforce(ciphertext):
    """
    Show all 26 possible Caesar decryptions.
    """
    ct = clean_text(ciphertext)
    results = []
    for key in range(26):
        pt = caesar_decrypt(ct, key)
        results.append((key, pt))
    return results


In [4]:
def multiplicative_encrypt(plaintext, key):
    pt = clean_text(plaintext)
    res = []
    for ch in pt:
        x = ALPHABET.index(ch)
        y = (key * x) % 26
        res.append(ALPHABET[y])
    return ''.join(res)

def multiplicative_decrypt(ciphertext, key):
    inv = mod_inverse(key, 26)
    if inv is None:
        raise ValueError("Key has no inverse mod 26.")
    ct = clean_text(ciphertext)
    res = []
    for ch in ct:
        y = ALPHABET.index(ch)
        x = (inv * y) % 26
        res.append(ALPHABET[x])
    return ''.join(res)

def affine_encrypt(plaintext, a, b):
    """
    E(x) = (a * x + b) mod 26
    """
    if mod_inverse(a, 26) is None:
        raise ValueError("a must be coprime to 26.")
    pt = clean_text(plaintext)
    res = []
    for ch in pt:
        x = ALPHABET.index(ch)
        y = (a * x + b) % 26
        res.append(ALPHABET[y])
    return ''.join(res)

def affine_decrypt(ciphertext, a, b):
    """
    D(y) = a^{-1} * (y - b) mod 26
    """
    inv_a = mod_inverse(a, 26)
    if inv_a is None:
        raise ValueError("a must be coprime to 26.")
    ct = clean_text(ciphertext)
    res = []
    for ch in ct:
        y = ALPHABET.index(ch)
        x = (inv_a * (y - b)) % 26
        res.append(ALPHABET[x])
    return ''.join(res)


In [5]:
def vigenere_encrypt(plaintext, key):
    pt = clean_text(plaintext)
    key = clean_text(key)
    res = []
    for i, ch in enumerate(pt):
        k = key[i % len(key)]
        shift = ALPHABET.index(k)
        idx = (ALPHABET.index(ch) + shift) % 26
        res.append(ALPHABET[idx])
    return ''.join(res)

def vigenere_decrypt(ciphertext, key):
    ct = clean_text(ciphertext)
    key = clean_text(key)
    res = []
    for i, ch in enumerate(ct):
        k = key[i % len(key)]
        shift = ALPHABET.index(k)
        idx = (ALPHABET.index(ch) - shift) % 26
        res.append(ALPHABET[idx])
    return ''.join(res)

def autokey_encrypt(plaintext, key):
    """
    Autokey using plaintext as extension of the key.
    """
    pt = clean_text(plaintext)
    key = clean_text(key) + pt  # key followed by plaintext
    res = []
    for i, ch in enumerate(pt):
        k = key[i]
        shift = ALPHABET.index(k)
        idx = (ALPHABET.index(ch) + shift) % 26
        res.append(ALPHABET[idx])
    return ''.join(res)

def autokey_decrypt(ciphertext, key):
    """
    Autokey decryption where key is extended by recovered plaintext.
    """
    ct = clean_text(ciphertext)
    key = list(clean_text(key))
    res = []
    for i, ch in enumerate(ct):
        if i < len(key):
            k = key[i]
        else:
            k = res[i - len(key)]  # previously recovered plaintext
        shift = ALPHABET.index(k)
        idx = (ALPHABET.index(ch) - shift) % 26
        pch = ALPHABET[idx]
        res.append(pch)
    return ''.join(res)


In [6]:
def playfair_generate_key_square(key):
    key = clean_text(key).replace("J", "I")
    used = set()
    square = []
    for ch in key + ALPHABET:
        if ch == 'J':
            continue
        if ch not in used:
            used.add(ch)
            square.append(ch)
    # 5x5 matrix
    matrix = [square[i:i+5] for i in range(0, 25, 5)]
    return matrix

def playfair_find_pos(matrix, ch):
    if ch == 'J':
        ch = 'I'
    for r in range(5):
        for c in range(5):
            if matrix[r][c] == ch:
                return r, c
    raise ValueError("Char not in matrix")

def playfair_prepare_text(text):
    text = clean_text(text).replace("J", "I")
    i = 0
    pairs = []
    while i < len(text):
        a = text[i]
        if i + 1 < len(text):
            b = text[i+1]
            if a == b:
                pairs.append(a + 'X')
                i += 1
            else:
                pairs.append(a + b)
                i += 2
        else:
            pairs.append(a + 'X')
            i += 1
    return pairs

def playfair_encrypt(plaintext, key):
    matrix = playfair_generate_key_square(key)
    pairs = playfair_prepare_text(plaintext)
    res = []
    for a, b in pairs:
        r1, c1 = playfair_find_pos(matrix, a)
        r2, c2 = playfair_find_pos(matrix, b)
        if r1 == r2:
            res.append(matrix[r1][(c1 + 1) % 5])
            res.append(matrix[r2][(c2 + 1) % 5])
        elif c1 == c2:
            res.append(matrix[(r1 + 1) % 5][c1])
            res.append(matrix[(r2 + 1) % 5][c2])
        else:
            res.append(matrix[r1][c2])
            res.append(matrix[r2][c1])
    return ''.join(res)

def playfair_decrypt(ciphertext, key):
    matrix = playfair_generate_key_square(key)
    res = []
    for i in range(0, len(ciphertext), 2):
        a = ciphertext[i]
        b = ciphertext[i+1]
        r1, c1 = playfair_find_pos(matrix, a)
        r2, c2 = playfair_find_pos(matrix, b)
        if r1 == r2:
            res.append(matrix[r1][(c1 - 1) % 5])
            res.append(matrix[r2][(c2 - 1) % 5])
        elif c1 == c2:
            res.append(matrix[(r1 - 1) % 5][c1])
            res.append(matrix[(r2 - 1) % 5][c2])
        else:
            res.append(matrix[r1][c2])
            res.append(matrix[r2][c1])
    return ''.join(res)


In [7]:
def column_order_from_key(key):
    key = key.upper()
    pairs = sorted([(ch, i) for i, ch in enumerate(key)])
    order = [None] * len(key)
    for new_pos, (_, old_pos) in enumerate(pairs):
        order[old_pos] = new_pos
    return order

def columnar_encrypt(plaintext, key):
    key = key.upper()
    order = column_order_from_key(key)
    pt = clean_text(plaintext)
    cols = len(key)
    rows = math.ceil(len(pt) / cols)
    # pad with X
    pt = pt.ljust(rows * cols, 'X')
    matrix = [pt[i*cols:(i+1)*cols] for i in range(rows)]
    # read columns in order
    ciphertext = ""
    for col_index in sorted(range(cols), key=lambda c: order[c]):
        for r in range(rows):
            ciphertext += matrix[r][col_index]
    return ciphertext

def columnar_decrypt(ciphertext, key):
    key = key.upper()
    order = column_order_from_key(key)
    cols = len(key)
    rows = math.ceil(len(ciphertext) / cols)
    # prepare empty matrix
    matrix = [[''] * cols for _ in range(rows)]
    # fill by columns in order
    c_index = 0
    for col_index in sorted(range(cols), key=lambda c: order[c]):
        for r in range(rows):
            if c_index < len(ciphertext):
                matrix[r][col_index] = ciphertext[c_index]
                c_index += 1
    # read row-wise
    plaintext = "".join("".join(row) for row in matrix)
    return plaintext


In [8]:
BAUDOT_ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ "

def make_baudot_tables():
    enc = {}
    dec = {}
    for i, ch in enumerate(BAUDOT_ALPHABET):
        code = format(i, '05b')  # 5-bit binary string
        enc[ch] = code
        dec[code] = ch
    return enc, dec

BAUDOT_ENC, BAUDOT_DEC = make_baudot_tables()

def baudot_encode(text):
    """
    Encode text (A–Z and space) into a 5-bit string.
    Other characters are ignored.
    """
    bits = ""
    for ch in text.upper():
        if ch in BAUDOT_ENC:
            bits += BAUDOT_ENC[ch]
    return bits

def baudot_decode(bits):
    """
    Decode a 5-bit string back into text using the simplified Baudot-like code.
    """
    res = []
    for i in range(0, len(bits), 5):
        chunk = bits[i:i+5]
        if len(chunk) < 5:
            break
        res.append(BAUDOT_DEC.get(chunk, '?'))
    return ''.join(res)

def xor_bitstrings(a, b):
    """
    XOR two bit strings up to the length of the shorter one.
    """
    L = min(len(a), len(b))
    return ''.join('1' if a[i] != b[i] else '0' for i in range(L))

def baudot_known_plain_attack(cipher1_bits, cipher2_bits, known_plain1):
    """
    Very simple demonstration of known-plaintext attack on two-time pad.

    We assume:
    - cipher1_bits = plaintext1_bits XOR key
    - cipher2_bits = plaintext2_bits XOR key
    - known_plain1 is the full plaintext1 (for demo)

    Then:
    key_bits = cipher1_bits XOR plaintext1_bits
    plaintext2_bits = cipher2_bits XOR key_bits
    """
    p1_bits = baudot_encode(known_plain1)
    # truncate to match min length
    L = min(len(cipher1_bits), len(p1_bits), len(cipher2_bits))
    cipher1_bits = cipher1_bits[:L]
    cipher2_bits = cipher2_bits[:L]
    p1_bits = p1_bits[:L]

    key_bits = xor_bitstrings(cipher1_bits, p1_bits)
    p2_bits = xor_bitstrings(cipher2_bits, key_bits)
    recovered_plain2 = baudot_decode(p2_bits)
    return recovered_plain2


In [9]:
def demo_baudot_two_time_pad():
    print("=== Baudot XOR Two-Time Pad Demo ===")
    p1 = input("Enter plaintext message 1 (A–Z and space): ")
    p2 = input("Enter plaintext message 2 (A–Z and space): ")

    p1_bits = baudot_encode(p1)
    p2_bits = baudot_encode(p2)

    # For simplicity, use a random key of same length as the shorter message
    import random
    key_length = min(len(p1_bits), len(p2_bits))
    key_bits = ''.join(random.choice('01') for _ in range(key_length))

    c1_bits = xor_bitstrings(p1_bits[:key_length], key_bits)
    c2_bits = xor_bitstrings(p2_bits[:key_length], key_bits)

    print("\n[Step 1] Ciphertexts (Baudot bits):")
    print("C1 bits:", c1_bits)
    print("C2 bits:", c2_bits)

    print("\n[Step 2] Attacker computes C1 XOR C2 = P1 XOR P2")
    p1_xor_p2_bits = xor_bitstrings(c1_bits, c2_bits)
    print("P1 XOR P2 bits:", p1_xor_p2_bits)

    print("\n[Step 3] Known-plaintext attack: assume attacker knows P1 exactly.")
    recovered_p2 = baudot_known_plain_attack(c1_bits, c2_bits, p1)
    print("Original P2:  ", p2.upper())
    print("Recovered P2: ", recovered_p2)
    print("\nThis shows why reusing a one-time pad key is insecure.")


In [10]:
!pip install ipywidgets --quiet

import ipywidgets as widgets
from IPython.display import display, clear_output


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.8 MB/s eta 0:00:00


In [11]:
# === GUI for Classical Ciphers ===

cipher_dropdown = widgets.Dropdown(
    options=[
        ('Caesar', 'caesar'),
        ('Multiplicative', 'mult'),
        ('Affine', 'affine'),
        ('Vigenere', 'vigenere'),
        ('Autokey', 'autokey'),
        ('Playfair', 'playfair'),
        ('Columnar Transposition', 'columnar')
    ],
    description='Cipher:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

mode_dropdown = widgets.Dropdown(
    options=[('Encrypt', 'E'), ('Decrypt', 'D'), ('Bruteforce (Caesar only)', 'B')],
    value='E',
    description='Mode:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

plaintext_area = widgets.Textarea(
    value='',
    placeholder='Enter text here...',
    description='Input Text:',
    layout=widgets.Layout(width='600px', height='80px'),
    style={'description_width': 'initial'}
)

key_box_1 = widgets.Text(
    value='',
    description='Key 1:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

key_box_2 = widgets.Text(
    value='',
    description='Key 2 (for Affine b or Columnar key, etc):',
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='Run Cipher',
    button_style='primary'
)

output_area = widgets.Output()


def run_cipher_button_clicked(b):
    with output_area:
        clear_output()
        cipher = cipher_dropdown.value
        mode = mode_dropdown.value
        text = plaintext_area.value

        try:
            if cipher == 'caesar':
                if mode == 'B':
                    results = caesar_bruteforce(text)
                    for key, pt in results:
                        print(f"Key {key:2d}: {pt}")
                    return
                key = int(key_box_1.value)
                if mode == 'E':
                    print("Ciphertext:", caesar_encrypt(text, key))
                else:
                    print("Plaintext:", caesar_decrypt(text, key))

            elif cipher == 'mult':
                key = int(key_box_1.value)
                if mode == 'E':
                    print("Ciphertext:", multiplicative_encrypt(text, key))
                else:
                    print("Plaintext:", multiplicative_decrypt(text, key))

            elif cipher == 'affine':
                a = int(key_box_1.value)
                b = int(key_box_2.value)
                if mode == 'E':
                    print("Ciphertext:", affine_encrypt(text, a, b))
                else:
                    print("Plaintext:", affine_decrypt(text, a, b))

            elif cipher == 'vigenere':
                key = key_box_1.value
                if mode == 'E':
                    print("Ciphertext:", vigenere_encrypt(text, key))
                else:
                    print("Plaintext:", vigenere_decrypt(text, key))

            elif cipher == 'autokey':
                key = key_box_1.value
                if mode == 'E':
                    print("Ciphertext:", autokey_encrypt(text, key))
                else:
                    print("Plaintext:", autokey_decrypt(text, key))

            elif cipher == 'playfair':
                key = key_box_1.value
                if mode == 'E':
                    print("Ciphertext:", playfair_encrypt(text, key))
                else:
                    print("Plaintext:", playfair_decrypt(text, key))

            elif cipher == 'columnar':
                key = key_box_1.value
                if mode == 'E':
                    print("Ciphertext:", columnar_encrypt(text, key))
                else:
                    print("Plaintext:", columnar_decrypt(text, key))

        except Exception as e:
            print("Error:", e)


run_button.on_click(run_cipher_button_clicked)

classical_gui = widgets.VBox([
    cipher_dropdown,
    mode_dropdown,
    widgets.HBox([plaintext_area]),
    widgets.HBox([key_box_1, key_box_2]),
    run_button,
    output_area
])


# === GUI for Baudot & XOR ===

baudot_input_text = widgets.Textarea(
    value='',
    placeholder='Enter A–Z and spaces...',
    description='Text:',
    layout=widgets.Layout(width='600px', height='80px'),
    style={'description_width': 'initial'}
)

baudot_bits_text = widgets.Textarea(
    value='',
    placeholder='Enter 5-bit string...',
    description='Bits:',
    layout=widgets.Layout(width='600px', height='80px'),
    style={'description_width': 'initial'}
)

baudot_encode_button = widgets.Button(
    description='Encode to Baudot',
    button_style=''
)

baudot_decode_button = widgets.Button(
    description='Decode from Baudot',
    button_style=''
)

xor_bits_1 = widgets.Textarea(
    value='',
    placeholder='First bit string...',
    description='Bits 1:',
    layout=widgets.Layout(width='600px', height='80px'),
    style={'description_width': 'initial'}
)

xor_bits_2 = widgets.Textarea(
    value='',
    placeholder='Second bit string...',
    description='Bits 2:',
    layout=widgets.Layout(width='600px', height='80px'),
    style={'description_width': 'initial'}
)

xor_button = widgets.Button(
    description='XOR Bit Strings',
    button_style=''
)

baudot_demo_button = widgets.Button(
    description='Run Two-Time Pad Demo',
    button_style='warning'
)

baudot_output = widgets.Output()


def baudot_encode_clicked(b):
    with baudot_output:
        clear_output()
        text = baudot_input_text.value
        bits = baudot_encode(text)
        print("Baudot bits:")
        print(bits)

def baudot_decode_clicked(b):
    with baudot_output:
        clear_output()
        bits = baudot_bits_text.value.strip()
        text = baudot_decode(bits)
        print("Decoded text:")
        print(text)

def xor_button_clicked(b):
    with baudot_output:
        clear_output()
        b1 = xor_bits_1.value.strip()
        b2 = xor_bits_2.value.strip()
        x = xor_bitstrings(b1, b2)
        print("XOR result:")
        print(x)

def baudot_demo_clicked(b):
    with baudot_output:
        clear_output()
        demo_baudot_two_time_pad()

baudot_encode_button.on_click(baudot_encode_clicked)
baudot_decode_button.on_click(baudot_decode_clicked)
xor_button.on_click(xor_button_clicked)
baudot_demo_button.on_click(baudot_demo_clicked)

baudot_gui = widgets.VBox([
    widgets.Label("Baudot Encode / Decode"),
    baudot_input_text,
    baudot_encode_button,
    baudot_bits_text,
    baudot_decode_button,
    widgets.Label("XOR Two Bit Strings"),
    xor_bits_1,
    xor_bits_2,
    xor_button,
    baudot_demo_button,
    baudot_output
])


# === TABS ===

tabs = widgets.Tab(children=[classical_gui, baudot_gui])
tabs.set_title(0, 'Classical Ciphers')
tabs.set_title(1, 'Baudot & XOR Tools')

display(tabs)
